## Analysis of Podcast Reviews

<b> Group 8A </b>: Site Bai, Kevin Cheung, Mira Daya, Ankit Jain, Yu Hsin (Kathy) Lee

In [ ]:
# Libraries
import os
import sqlite3 as sq
import pandas as pd
from collections import defaultdict
#!pip install wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import re
import datetime
import numpy as np
from matplotlib import rc
import seaborn as sns
#!pip install plotnine
from statsmodels.tsa.arima_process import ArmaProcess
!pip install pycausalimpact --user
#from causalimpact import CausalImpact
from plotnine import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('stopwords')
stopwords.words('english')
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
import string
from textblob import TextBlob

In [ ]:
# Importing Data
os.chdir('C:/Users/jaina/Downloads') # Changing Directory according to computer
connection = sq.connect('database.sqlite')
cur = connection.cursor()

In [ ]:
# Categories Dataset
cur.execute('SELECT * FROM categories')
categories = defaultdict(list)   
for row in cur.fetchall():
    categories['Podcast_ID'].append(row[0])
    categories['Category'].append(row[1])
    
categories = pd.DataFrame(categories, columns = list(categories.keys()))
categories

In [ ]:
len(list(pd.DataFrame(categories['Category'].value_counts()).index))

In [ ]:
# Podcasts Dataset
cur.execute('SELECT * FROM podcasts')
podcasts = defaultdict(list)   
for row in cur.fetchall():
    podcasts['Podcast_ID'].append(row[0])
    podcasts['Itunes_ID'].append(row[1])
    podcasts['Slug'].append(row[2])
    podcasts['Itunes_URL'].append(row[3])
    podcasts['Title'].append(row[4])
    
podcasts = pd.DataFrame(podcasts, columns = list(podcasts.keys()))
podcasts

In [ ]:
def podname_finder(given_id):
    return list(podcasts[podcasts.Podcast_ID == given_id]['Title'])

In [ ]:
# Reviews Dataset
cur.execute('SELECT * FROM reviews')
reviews = defaultdict(list)   
for row in cur.fetchall():
    reviews['Podcast_ID'].append(row[0])
    reviews['Review_Title'].append(row[1])
    reviews['Review_Description'].append(row[2])
    reviews['Review_Rating'].append(int(row[3]))
    reviews['Review_Created_At'].append(row[4])
    
reviews = pd.DataFrame(reviews, columns = list(reviews.keys()))
reviews

In [ ]:
reviews.isnull().sum().to_frame()

In [ ]:
categories.isnull().sum().to_frame()

In [ ]:
podcasts.isnull().sum().to_frame()

## Exploratory Data Analysis

In [ ]:
plt.bar([str(x) for x in list(pd.DataFrame(reviews['Review_Rating'].value_counts()).index)], 
         list(reviews['Review_Rating'].value_counts()))
plt.title('Number of Reviews by Rating')
plt.xlabel('Review Rating')
plt.ylabel('Review Count')
plt.show()

In [ ]:
# Transforming the Review Created At Variable
list1 = []
for time in reviews['Review_Created_At']:
    t = re.sub(r"-07:00", "", time)
    t = re.sub(r"T", " ", t)
    list1.append(t)

reviews['Review_DateTime'] = list1

# Keep rows that contain numbers
reviews = reviews[reviews['Review_DateTime'].str.contains(r'[0-9]+')]

# Transform to datetime object
for time in reviews['Review_DateTime']:
    t = datetime.datetime.strptime(time,"%Y-%m-%d %H:%M:%S") 
    datetime.datetime.timestamp(t) 

In [ ]:
# Extract year, month, hour, and weekday from DateTime
reviews["Review_DateTime"] = pd.to_datetime(reviews["Review_DateTime"])
reviews["Year"] = reviews["Review_DateTime"].dt.year
reviews["Month"] = reviews["Review_DateTime"].dt.month
reviews["Weekday"] = reviews["Review_DateTime"].dt.weekday
reviews["Hour"] = reviews["Review_DateTime"].dt.hour

In [ ]:
# Reviews Over Time from 2006 to 2020
(ggplot(reviews, aes("Review_DateTime")) 
 + geom_area(stat = "bin", fill = "cornflowerblue") 
 + theme(axis_text_x = element_text(angle = 45)) 
 + labs(x = "Date", y = "Count", title = "Reviews Over Time"))

### By Year (2011-2020)

In [ ]:
# Extract rows between 2015-2021 and group them by review ratings
year = reviews[reviews['Year'] >= 2011]
#year.to_csv(r'/Users/KathyLee/UCI MSBA/Course Material/04 Winter Quarter/BANA277 Customer & Social Analytics/Project/ Reviews2011.csv', index = False)

# Group by Rating and Year
ratingbyyear = year.groupby(["Review_Rating", "Year"])["Review_Rating"].count()

In [ ]:
# Percentage of Reviews Ratings by Year (2011-2020)
plt.figure(figsize = (9,6))

# Values of each group
bars1 = [ratingbyyear[1][2011], ratingbyyear[1][2012], ratingbyyear[1][2013], ratingbyyear[1][2014], ratingbyyear[1][2015], ratingbyyear[1][2016], ratingbyyear[1][2017], ratingbyyear[1][2018], ratingbyyear[1][2019], ratingbyyear[1][2020]]
bars2 = [ratingbyyear[2][2011], ratingbyyear[2][2012], ratingbyyear[2][2013], ratingbyyear[2][2014], ratingbyyear[2][2015], ratingbyyear[2][2016], ratingbyyear[2][2017], ratingbyyear[2][2018], ratingbyyear[2][2019], ratingbyyear[2][2020]]
bars3 = [ratingbyyear[3][2011], ratingbyyear[3][2012], ratingbyyear[3][2013], ratingbyyear[3][2014], ratingbyyear[3][2015], ratingbyyear[3][2016], ratingbyyear[3][2017], ratingbyyear[3][2018], ratingbyyear[3][2019], ratingbyyear[3][2020]]
bars4 = [ratingbyyear[4][2011], ratingbyyear[4][2012], ratingbyyear[4][2013], ratingbyyear[4][2014], ratingbyyear[4][2015], ratingbyyear[4][2016], ratingbyyear[4][2017], ratingbyyear[4][2018], ratingbyyear[4][2019], ratingbyyear[4][2020]]
bars5 = [ratingbyyear[5][2011], ratingbyyear[5][2012], ratingbyyear[5][2013], ratingbyyear[5][2014], ratingbyyear[5][2015], ratingbyyear[5][2016], ratingbyyear[5][2017], ratingbyyear[5][2018], ratingbyyear[5][2019], ratingbyyear[5][2020]]

totals = [i+j+k+z+g for i,j,k,z,g in zip(bars1, bars2, bars3, bars4, bars5)]
One = [i / j * 100 for i,j in zip(bars1, totals)]
Two = [i / j * 100 for i,j in zip(bars2, totals)]
Three = [i / j * 100 for i,j in zip(bars3, totals)]
Four = [i / j * 100 for i,j in zip(bars4, totals)]
Five = [i / j * 100 for i,j in zip(bars5, totals)]


# The position of the bars on the x-axis
r = [0,1,2,3,4,5,6,7,8,9]
 
# Names of group and bar width
names = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
barWidth = 1

# Create bars
plt.bar(r, One, color = 'slategrey', edgecolor = 'white', width = barWidth)
plt.bar(r, Two, bottom = One, color='lightsteelblue', edgecolor = 'white', width = barWidth)
plt.bar(r, Three, bottom = [i+j for i,j in zip(One,Two)], color = 'cornflowerblue', edgecolor = 'white', width = barWidth)
plt.bar(r, Four, bottom = [i+j+z for i,j,z in zip(One,Two,Three)], color = 'royalblue', edgecolor = 'white', width = barWidth)
plt.bar(r, Five, bottom = [i+j+z+g for i,j,z,g in zip(One, Two, Three, Four)], color = 'midnightblue', edgecolor = 'white', width = barWidth)

# Aes
plt.xticks(r, names)
plt.xlabel("Year", weight = "bold", size = 14)
plt.ylabel("Percentage", weight = "bold", size = 14)
plt.legend(['1','2','3','4','5'], fontsize = 12)
plt.title("Percentage of Review Ratings by Year", weight = "bold", size = 16)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

# Show graphic
plt.show()

### By Hour

In [ ]:
rvcnth = pd.DataFrame(reviews.groupby("Hour", as_index = False)["Hour"].count())
rvcnth = rvcnth.rename(columns = {"Hour": "Count"})
rvcnth["Hour"] = ['12 pm', '1 am', '2 am', '3 am', '4  am', '5  am', '6 am', '7 am', '8 am', '9 am', '10 am', '11 am', '12 am', '1 pm', '2 pm', '3 pm', '4 pm', '5 pm', '6 pm', '7 pm', '8 pm', '9 pm', '10 pm', '11 pm']

# Barplot
plt.figure(figsize = (22,13))

barplot = plt.bar(rvcnth["Hour"], rvcnth["Count"], color = "midnightblue", width = 0.7)
barplot[10].set_color('orange')
barplot[1].set_color('orange')

plt.title('Review Count by Hours', fontsize = 30, weight = "bold")
plt.xlabel('Hours', fontsize = 18, weight = "bold")
plt.ylabel('Count', fontsize = 18, weight = "bold")
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.show()

In [ ]:
# Average Reviews Rating by Hours
avgrt = reviews[["Hour", "Review_Rating"]]
avgrt = avgrt.groupby("Hour", as_index = False).mean()
avgrt["Hour"] = ['12 pm', '1 am', '2 am', '3 am', '4  am', '5  am', '6 am', '7 am', '8 am', '9 am', '10 am', '11 am', '12 am', '1 pm', '2 pm', '3 pm', '4 pm', '5 pm', '6 pm', '7 pm', '8 pm', '9 pm', '10 pm', '11 pm']

with plt.style.context('ggplot'):
    plt.figure(figsize = (20,10))
    plt.plot(avgrt["Hour"], avgrt["Review_Rating"], linestyle='solid', linewidth = 5, color = "cornflowerblue")
    plt.title('Average Review Rating by Hours', fontsize = 30, weight = "bold")
    plt.xlabel('Hours', fontsize = 18, weight = "bold")
    plt.ylabel('Review_Rating', fontsize = 18, weight = "bold")
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
plt.show()

### By Weekday

In [ ]:
reviewsbyweekday = pd.DataFrame(reviews.groupby("Weekday", as_index = False)["Weekday"].count())
reviewsbyweekday = reviewsbyweekday.rename(columns = {"Weekday": "Count"})
reviewsbyweekday["Weekday"] = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]

# Barplot
plt.figure(figsize = (9, 6))
barplot = plt.bar(reviewsbyweekday["Weekday"], reviewsbyweekday["Count"], color = "midnightblue", width = 0.7)
barplot[2].set_color('orange')
barplot[5].set_color('orange')

plt.title('Review Count by Weekday', fontsize = 16, weight = "bold")
plt.xlabel('Weekday', fontsize = 14, weight = "bold")
plt.ylabel('Count', fontsize = 14, weight = "bold")
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.show()

In [ ]:
# Average Review Rating by Weekdays
week = reviews[["Weekday", "Review_Rating"]]
avgrw = week.groupby("Weekday", as_index = False).mean()
avgrw['Weekday'] = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]

with plt.style.context('ggplot'):
    plt.figure(figsize = (10,8))
    plt.plot(avgrw["Weekday"], avgrw["Review_Rating"], linestyle='solid', linewidth = 5, color = "cornflowerblue")
    plt.title('Average Review Rating by Weekday', fontsize = 20, weight = "bold")
    plt.xlabel('Weekday', fontsize = 16, weight = "bold")
    plt.ylabel('Average Review_Rating', fontsize = 16, weight = "bold")
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)    
plt.show()

In [ ]:
# Review Rating over the Years
# The trend of number of rating over the study period from year 2006 to year 2020 since the year 2021 is not over yet.
no_2105 = reviews[(reviews.Year != 2021) &  (reviews.Year != 2005)]
(ggplot(no_2105.groupby('Year')['Review_Rating'].count().reset_index()) +
    aes('Year', 'Review_Rating') +
    geom_line())

In [ ]:
one_five = no_2105.groupby(['Year','Review_Rating'])['Podcast_ID'].count()
one_five_props = one_five.groupby(level = 0).apply(lambda x :  100 * x / float(x.sum()))
one_five_props = one_five_props.reset_index()
ones = one_five_props[one_five_props['Review_Rating'] == 1]
ones = ones.rename(columns = {'Podcast_ID':'proportion'})
fives = one_five_props[one_five_props['Review_Rating'] == 5]
fives = fives.rename(columns = {'Podcast_ID':'proportion'})

# Prepare the dataframe for creating line plot with dual Y axis. 
ones['proportion_5'] = fives.proportion.values

In [ ]:
# As shown in the plot, in 2020, the proportion of rating 1 rised up to the highest point, and on the contrast, 
# the proportion of rating 5 dropped to the lowest point among past five years. 
fig,ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(ones['Year'],ones['proportion'], 'g-' )
ax2.plot(ones['Year'], ones['proportion_5'],'b-')
ax1.set_xlabel('Year (2006 to 2020)')
ax1.set_ylabel('Proportion of Rating 1', color='g')
ax2.set_ylabel('Proportion of Rating 5', color='b')
plt.show()

In [ ]:
cag_rev =  pd.merge(categories, reviews, on='Podcast_ID')
cag_rev['Review_Created_At'] = pd.to_datetime(cag_rev['Review_Created_At'],infer_datetime_format=True )
cag_rev['year'] = cag_rev['Review_Created_At'].dt.year
cag_rev['month'] = cag_rev['Review_Created_At'].dt.month
cag_rev['hour'] = cag_rev['Review_Created_At'].dt.hour

In [ ]:
# Seeing the most popular categories being reviewed
cag_rev['Category'].value_counts()

In [ ]:
top_seven = ['society-culture',"comedy","business","tv-film","education",'arts',"religion-spirituality"]
top_seven_table = cag_rev[cag_rev['Category'].isin(top_seven)]

In [ ]:
pandemic_year = top_seven_table[(top_seven_table.year == 2019) | (top_seven_table.year == 2020)]
pandemic_group = pandemic_year.groupby(['year','month','Review_Rating'])['Podcast_ID'].count()
with_categs = pandemic_year.groupby(['year','month','Category','Review_Rating'])['Podcast_ID'].count()
with_catehs_props = with_categs.groupby(level=[0,1,2]).apply(lambda x : 100 * x / float(x.sum()))
with_catehs_props = with_catehs_props.reset_index()
with_categ_new = with_catehs_props.rename(columns = {'Podcast_ID':'proportion'})
catehs_ones = with_categ_new[with_categ_new['Review_Rating'] == 1]
society_culture_p = catehs_ones[catehs_ones.Category == 'society-culture'].proportion.values
arts_p = catehs_ones[catehs_ones.Category == 'arts'].proportion.values
comedy_p = catehs_ones[catehs_ones.Category == 'comedy'].proportion.values
tv_film_p = catehs_ones[catehs_ones.Category == 'tv-film'].proportion.values
business_p = catehs_ones[catehs_ones.Category == 'business'].proportion.values
eductaion_p = catehs_ones[catehs_ones.Category == 'education'].proportion.values
religion_spirituality_p= catehs_ones[catehs_ones.Category == 'religion-spirituality'].proportion.values

In [ ]:
plot_df = pd.DataFrame({'x':range(1,25),
                      'society':society_culture_p,
                      'arts':arts_p,
                       'comedy':comedy_p,
                       'tv_film': tv_film_p,
                        'business':business_p,
                        'education':eductaion_p,
                        'religion_spirituality':religion_spirituality_p
                       })

In [ ]:
# Proprotion of Rating 1 over Year 2019 & Year 2020
plt.figure(figsize=(20,10))
plt.plot(plot_df['x'], plot_df['society'], marker='', color='grey', linewidth=2, alpha=0.4)
plt.plot(plot_df['x'], plot_df['arts'], marker='', color='blue', linewidth=2, alpha=0.4)
plt.plot(plot_df['x'], plot_df['comedy'], marker='', color='orange', linewidth=2, alpha=0.4)
plt.plot(plot_df['x'], plot_df['tv_film'], marker='', color='green', linewidth=2, alpha=0.4)
plt.plot(plot_df['x'], plot_df['business'], marker='', color='pink', linewidth=2, alpha=0.4)
plt.plot(plot_df['x'], plot_df['education'], marker='', color='red', linewidth=2, alpha=0.4)
plt.plot(plot_df['x'], plot_df['religion_spirituality'], marker='', color='purple', linewidth=2, alpha=0.4)
plt.plot([18, 18], [0, 18], 'k-', linestyle='dashed', lw=1)
plt.plot([15, 15], [0, 18], 'k-', linestyle='dashed', lw=1)
plt.text(18.3, 10, 'June 2020', fontsize = 16, color = 'red')
plt.text(12.7, 12.5, 'March 2020', fontsize = 16, color = 'red')

plt.title('Proprotion of Rating 1 over Year 2019 & Year 2020', fontsize = 18)
plt.xlabel('2019-2020', fontsize = 18)
plt.ylabel('Proportions', fontsize = 18)


num = 0
for i in plot_df.values[23][1:]:
    num += 1
    name = list(plot_df)[num]
    plt.text(24, i, name, horizontalalignment='left',fontsize = 12, color='black')

In [ ]:
catehs_fives = with_categ_new[with_categ_new['Review_Rating'] == 5]
society_culture_f = catehs_fives[catehs_fives.Category == 'society-culture'].proportion.values
arts_f = catehs_fives[catehs_fives.Category == 'arts'].proportion.values
comedy_f = catehs_fives[catehs_fives.Category == 'comedy'].proportion.values
tv_film_f = catehs_fives[catehs_fives.Category == 'tv-film'].proportion.values
business_f = catehs_fives[catehs_fives.Category == 'business'].proportion.values
eductaion_f = catehs_fives[catehs_fives.Category == 'education'].proportion.values
religion_spirituality_f= catehs_fives[catehs_fives.Category == 'religion-spirituality'].proportion.values

In [ ]:
plot_df_five = pd.DataFrame({'x':range(1,25),
                      'society':society_culture_f,
                      'arts':arts_f,
                       'comedy':comedy_f,
                       'tv_film': tv_film_f,
                        'business':business_f,
                        'education':eductaion_f,
                        'religion_spirituality':religion_spirituality_f
                       })

In [ ]:
# Proprotion of Rating 5 over Year 2019 & Year 2020
plt.figure(figsize=(20,10))
plt.plot(plot_df['x'], plot_df_five['society'], marker='', color='grey', linewidth=2, alpha=0.4)
plt.plot(plot_df['x'], plot_df_five['arts'], marker='', color='blue', linewidth=2, alpha=0.4)
plt.plot(plot_df['x'], plot_df_five['comedy'], marker='', color='orange', linewidth=2, alpha=0.4)
plt.plot(plot_df['x'], plot_df_five['tv_film'], marker='', color='green', linewidth=2, alpha=0.4)
plt.plot(plot_df['x'], plot_df_five['business'], marker='', color='pink', linewidth=2, alpha=0.4)
plt.plot(plot_df['x'], plot_df_five['education'], marker='', color='red', linewidth=2, alpha=0.4)
plt.plot(plot_df['x'], plot_df_five['religion_spirituality'], marker='', color='purple', linewidth=1, alpha=0.4)
plt.plot([18, 18], [70, 95], 'k-', linestyle='dashed', lw=1)

plt.text(18.2, 85, 'June 2020', fontsize = 18, color = 'red')
#plt.text(9, 85, 'March 2020', fontsize = 12, color = 'red')

plt.title('Proprotion of Rating 5 over Year 2019 & Year 2020')
plt.xlabel('2019-2020')
plt.ylabel('Proportions')

num = 0
for i in plot_df_five.values[23][1:]:
    num += 1
    name = list(plot_df_five)[num]
    plt.text(24, i, name, horizontalalignment='left',fontsize = 12, color='black')

In [ ]:
def podcat(given_id):
    '''
    Generalizing categories for a given podcast ID
    '''
    cat = list(categories[categories.Podcast_ID == given_id]['Category'])
    cat_dict = defaultdict(int)
    for c in cat:
        if 'arts' in c:
            cat_dict['arts'] += 1
        elif 'business' in c:
            cat_dict['business'] += 1
        elif 'society-culture' in c:
            cat_dict['society_culture'] += 1
        elif c in ['buddhism','christianity','hinduism','islam','judaism','religion-spirituality','spirituality']:
            cat_dict['religion-spirituality'] += 1
        else:
            cat_dict[c] += 1
    for k in cat_dict.keys():
        if cat_dict[k] == max(dict(cat_dict).values()):
            return k

In [ ]:
# Adding Category to Podcasts Table
podcasts['Category'] = [podcat(i) for i in podcasts['Podcast_ID']]

In [ ]:
# Adding Category to Reviews Table
reviews = reviews.merge(podcasts, on = 'Podcast_ID', how = 'right')[['Podcast_ID','Review_Title','Review_Description',
                                                                    'Review_Rating','Review_Created_At','Category']]

In [ ]:
# Bar Graph for Generalized Categories
plt.barh(list(pd.DataFrame(podcasts['Category'].value_counts()).index), 
         list(podcasts['Category'].value_counts()))
plt.title('Number of Podcasts by Generalized Category')
plt.xlabel('Number of Podcasts')
plt.ylabel('Category')
plt.show()

In [ ]:
plt.barh(list(pd.DataFrame(reviews['Category'].value_counts()).index), 
         list(reviews['Category'].value_counts()))
plt.title('Number of Reviews by General Category')
plt.xlabel('Number of Reviews')
plt.ylabel('Category')
plt.show()

In [ ]:
# Average Review Rating by General Category
pd.DataFrame(reviews.groupby('Category').mean()).sort_values('Review_Rating')

### Text Modelling

In [ ]:
def rating_clouds(given_rating):
    stopwords = set(STOPWORDS)
    stopwords.update(['podcast', 'listen', 'podcasts', 'listening','episode', 'listened','s',
                      'show', 'episodes', 'used', 'good', 'love', 'listener', 'will', 'really'
                     , 'like', 'people', 'don', 't', 'm', '&','ve', 're', 'one'])

    ratings_table = reviews[reviews.Review_Rating == given_rating]
    description_text = ' '.join(review for review in ratings_table.Review_Description)
    description_text = description_text.lower()
    description_text = description_text.strip()

    title_text = ' '.join(review for review in ratings_table.Review_Title)
    title_text = title_text.strip()
    
    print('Rating: {}'.format(given_rating))
    print('Word Cloud for Review Descriptions')
    plt.figure(figsize=(20, 20))
    plt.imshow(WordCloud(stopwords = stopwords).generate(description_text), interpolation = 'bilinear')
    plt.axis('off')
    plt.show()
    
    print('Word Cloud for Review Titles')
    plt.figure(figsize=(20, 20))
    plt.imshow(WordCloud(stopwords = stopwords).generate(title_text), interpolation = 'bilinear')
    plt.axis('off')
    plt.show()

In [ ]:
for rating in sorted(set(reviews['Review_Rating'])):
    rating_clouds(rating)

In [ ]:
# Function for tokenization including stemming 
from nltk.stem.porter import PorterStemmer

# Initialize stemmer using porterstemmer
stemmer = PorterStemmer()

stopwords_list = []

stopwords = set(stopwords.words('english'))
stopwords1 = stopwords.union(set(stopwords_list)) # stopwords into list

def tokenize_stem(text):
    try: 
        regex = re.compile('[' +re.escape(string.punctuation) + '0-9\\r\\t\\n]') 
        text = regex.sub(" ", text) 
        
        tokens1 = [word_tokenize(s) for s in sent_tokenize(text)] 
        tokens = []
        for token_by_sent in tokens1:
            tokens += token_by_sent
        tokens = list(filter(lambda t: t.lower() not in stopwords1, tokens))
        new_tokens = [w for w in tokens if re.search('[a-zA-Z]', w)]
        new_tokens = [w.lower() for w in new_tokens if len(w)>=3]
        
        # Stemming 
        stems = [stemmer.stem(item) for item in new_tokens]
        return stems
    
    except TypeError as e: print(text,e)

In [ ]:
# Function to print top words in topic modeling
def print_top_words(model, feature_names, n_top_words):
    for index, topic in enumerate(model.components_): #model.components_ gives an array (n_components, n_features)
        message = "Topic #{}:".format(index)
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1 :-1]])
        print(message)
        print("_"*70) 

In [ ]:
# Considering Data only after 2020 for analysis
cur.execute('SELECT * FROM reviews')
reviews = defaultdict(list)   
for row in cur.fetchall():
    reviews['Podcast_ID'].append(row[0])
    reviews['Review_Title'].append(row[1])
    reviews['Review_Description'].append(row[2])
    reviews['Review_Rating'].append(int(row[3]))
    reviews['Review_Created_At'].append(row[4])    
reviews = pd.DataFrame(reviews, columns = list(reviews.keys()))
reviews['Review_Created_At'] = pd.to_datetime(reviews['Review_Created_At'])
reviews = reviews[pd.DatetimeIndex(reviews['Review_Created_At']).year == 2020].reset_index(drop = True)
reviews['month'] = pd.DatetimeIndex(reviews['Review_Created_At']).month

In [ ]:
# Tokenizing Description 
reviews['desc_stem'] = reviews['Review_Description'].map(tokenize_stem)
reviews.reset_index(drop=True, inplace=True)

In [ ]:
# Creating Tf-Idf Scores for Review Descriptions
tfidf_vec = TfidfVectorizer(min_df=0.005,
                             max_features=None,
                             tokenizer=tokenize_stem,
                             ngram_range=(2, 3))
tfidf_e = reviews['Review_Description'].values
tfidf_vz = tfidf_vec.fit_transform(list(tfidf_e))

In [ ]:
tfidf_table = dict(zip(tfidf_vec.get_feature_names(), tfidf_vec.idf_))

tfidf_table = pd.DataFrame(columns=['tfidf']).from_dict(dict(tfidf_table), orient='index')
tfidf_table.columns = ['tfidf']

In [ ]:
# Lower the TF IDF score, the more common the word 
tfidf_table.sort_values(by = ['tfidf'], ascending = True).head(20)

In [ ]:
# Higher the TF IDF score, the more rare the terms
tfidf_table.sort_values(by = ['tfidf'], ascending = False).head(20)

In [ ]:
# Clustering Each Review
# NMF with TfidfVectorizer 
n_top_words = 15

nmf1 = NMF(init="nndsvd",
            n_components=20,
            max_iter=200, random_state=0)
A1 = tfidf_vz
W1 = nmf1.fit_transform(A1) 
H1 = nmf1.components_ 

tfidf_feature_names = tfidf_vec.get_feature_names()
print_top_words(nmf1, tfidf_feature_names, n_top_words)

### Sentiment Analysis

In [ ]:
def getSubjectivity(text): # Function to get Subjectivity
    return TextBlob(text).sentiment.subjectivity

In [ ]:
def getPolarity(text): # Function to get Polarity
    return TextBlob(text).sentiment.polarity

In [ ]:
def getAnalysis(score):
    if score < 0:
        return "Negative"
    elif score == 0:
        return "Neutral"
    else:
        return "Positive"

In [ ]:
def sentiment_analyzer(df, col):
    df["TextBlob_Subjectivity_{}".format(col)] = df[col].apply(getSubjectivity)
    df["TextBlob_Polarity_{}".format(col)] = df[col].apply(getPolarity)
    df["TextBlob_Analysis_{}".format(col)] = df["TextBlob_Polarity_{}".format(col)].apply(getAnalysis)

In [ ]:
# Getting the sentiment of Review Description and Title
sentiment_analyzer(reviews, 'Review_Description')
sentiment_analyzer(reviews, 'Review_Title')

In [ ]:
# Polarity throughout 2020
plt.figure(figsize=(9,4))
sns.lineplot(x = 'month', y = 'TextBlob_Polarity_Review_Description', data = reviews).set_title('Polarity Across 2020 (Description)')
plt.show()

plt.figure(figsize=(9,4))
sns.lineplot(x = 'month', y = 'TextBlob_Polarity_Review_Title', data = reviews).set_title('Polarity Across 2020 (Title)')
plt.show()

In [ ]:
plt.bar(['Positive', 'Neutral', 'Negative'], list(reviews['TextBlob_Analysis_Review_Description'].value_counts()), 
            color = ['orange', 'grey', 'blue'])
plt.title('Review Description Sentiments')
plt.ylabel('Number of Reviews')
plt.xlabel('Sentiment')
plt.show()

In [ ]:
plt.bar(['Positive', 'Neutral', 'Negative'], list(reviews['TextBlob_Analysis_Review_Title'].value_counts()), 
            color = ['orange', 'grey', 'blue'])
plt.title('Review Title Sentiments')
plt.ylabel('Number of Reviews')
plt.xlabel('Sentiment')
plt.show()

In [ ]:
# Count of Ratings for Negative Sentiment
sns.countplot(x = 'Review_Rating', data = reviews[reviews['TextBlob_Analysis_Review_Description'] == 'Negative']).set_title('Count of Ratings for Negative Sentiment')
plt.show()

In [ ]:
# Count of Ratings for Positive Sentiment
sns.countplot(x = 'Review_Rating', data = reviews[reviews['TextBlob_Analysis_Review_Description'] == 'Positive']).set_title('Count of Ratings for Positive Sentiment')
plt.show()

In [ ]:
# Count of Ratings for Neutral Sentiment
sns.countplot(x = 'Review_Rating', data = reviews[reviews['TextBlob_Analysis_Review_Description'] == 'Neutral']).set_title('Count of Ratings for Neutral Sentiment')
plt.show()

### Changes During Historic Events in 2020

In [ ]:
# Filtering Reviews based on some common words associated with COVID
covid = reviews[reviews['Review_Description'].str.contains('pandemic' , regex=False, case=False, na=False)
                | reviews['Review_Description'].str.contains('corona' , regex=False, case=False, na=False)
                | reviews['Review_Description'].str.contains('virus' , regex=False, case=False, na=False)
                | reviews['Review_Description'].str.contains('covid' , regex=False, case=False, na=False)
                | reviews['Review_Description'].str.contains('cdc' , regex=False, case=False, na=False)
                | reviews['Review_Description'].str.contains('rona' , regex=False, case=False, na=False)
                | reviews['Review_Description'].str.contains('fauci' , regex=False, case=False, na=False)
                | reviews['Review_Description'].str.contains('wfh' , regex=False, case=False, na=False)
                | reviews['Review_Description'].str.contains('distanc' , regex=False, case=False, na=False)
                | reviews['Review_Description'].str.contains('sheep' , regex=False, case=False, na=False)
                | reviews['Review_Description'].str.contains('mask' , regex=False, case=False, na=False)
                | reviews['Review_Description'].str.contains('vaccine' , regex=False, case=False, na=False)
                | reviews['Review_Description'].str.contains('pfizer' , regex=False, case=False, na=False)
                | reviews['Review_Description'].str.contains('moderna' , regex=False, case=False, na=False)]

In [ ]:
# Filtering Reviews based on some common words associated with the BLM Protests
blm = reviews[reviews['Review_Description'].str.contains('blm' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('black lives matter' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('blacklivesmatter' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('george floyd' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('racism' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('white' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('supremacy' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('trump' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('racist' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('brutality' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('breonna taylor' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('police' , regex=False, case=False, na=False)]

In [ ]:
# Filtering Reviews based on some common words associated with the Election and Politics
politics = reviews[reviews['Review_Description'].str.contains('election' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('president' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('trump' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('biden' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('democrat' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('republican' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('govt' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('liberal' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('conservative' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('left' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('right' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('harris' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('pence' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('politics' , regex=False, case=False, na=False)|
              reviews['Review_Description'].str.contains('media' , regex=False, case=False, na=False)]

In [ ]:
plt.bar(['Positive', 'Neutral', 'Negative'], list(covid['TextBlob_Analysis_Review_Description'].value_counts()), 
            color = ['orange', 'grey', 'blue'])
plt.title('Sentiments for COVID Related Data (Description)')
plt.ylabel('Number of Reviews')
plt.xlabel('Sentiment')
plt.show()

plt.bar(['Positive', 'Neutral', 'Negative'], list(covid['TextBlob_Analysis_Review_Title'].value_counts()), 
            color = ['orange', 'grey', 'blue'])
plt.title('Sentiments for COVID Related Data (Title)')
plt.ylabel('Number of Reviews')
plt.xlabel('Sentiment')
plt.show()

In [ ]:
plt.bar(['Positive', 'Neutral', 'Negative'], list(blm['TextBlob_Analysis_Review_Description'].value_counts()), 
            color = ['orange', 'grey', 'blue'])
plt.title('Sentiments for BLM Related Data (Description)')
plt.ylabel('Number of Reviews')
plt.xlabel('Sentiment')
plt.show()

plt.bar(['Positive', 'Neutral', 'Negative'], list(blm['TextBlob_Analysis_Review_Title'].value_counts()), 
            color = ['orange', 'grey', 'blue'])
plt.title('Sentiments for BLM Related Data (Title)')
plt.ylabel('Number of Reviews')
plt.xlabel('Sentiment')
plt.show()

In [ ]:
plt.bar(['Positive', 'Neutral', 'Negative'], list(politics['TextBlob_Analysis_Review_Description'].value_counts()), 
            color = ['orange', 'grey', 'blue'])
plt.title('Sentiments for Politics Related Data (Description)')
plt.ylabel('Number of Reviews')
plt.xlabel('Sentiment')
plt.show()

plt.bar(['Positive', 'Neutral', 'Negative'], list(politics['TextBlob_Analysis_Review_Title'].value_counts()), 
            color = ['orange', 'grey', 'blue'])
plt.title('Sentiments for Politics Related Data (Title)')
plt.ylabel('Number of Reviews')
plt.xlabel('Sentiment')
plt.show()

In [ ]:
blm_2020 = cag_rev[cag_rev['year'] == 2020]
blm_2020_month = blm_2020.groupby(['month','Review_Rating'])['Podcast_ID'].count()
blm_2020_props = blm_2020_month.groupby(level = 0).apply(lambda x : 100 * x / float(x.sum()))
blm_2020_props = blm_2020_props.reset_index()
blm_2020_props = blm_2020_props.rename(columns = {'Podcast_ID':'proportion'})
blm_one_props = blm_2020_props[blm_2020_props['Review_Rating'] == 1]
blm_five_props = blm_2020_props[blm_2020_props['Review_Rating'] == 5]

Using the historic Black lives Matter movement protests of 2020 as the treatment to fit the model

In [ ]:
# Data Prepration
blm = pd.read_csv("multiTimeline.csv")
blm = blm.reset_index()
blm.drop(0, inplace = True)
blm = blm.rename(columns = {'index': 'date', 'Category: All categories':'score'})
blm['date'] = pd.to_datetime(blm['date'], infer_datetime_format = True)
blm['year'] = blm['date'].dt.year
blm['month'] = blm['date'].dt.month
blm['day'] = blm['date'].dt.day
blm['score'] = blm['score'].astype('int')
avg_score_blm = blm.groupby('month')['score'].mean()
avg_score_blm = avg_score_blm.reset_index()

In [ ]:
!pip install pycausalimpact --user

In [ ]:
# The below summary shows that the BLM Protests did have a significant negative effect on the proportion of rating 1
y  = list(blm_one_props.proportion)
X  = list(avg_score_blm.score)
model_data = pd.DataFrame({'y': y, 'X':X}, columns = ['y', 'X'])

pre_period = [0,4]
post_period = [5, 11]
ci = CausalImpact(model_data, pre_period, post_period)
print(ci.summary())
print(ci.summary(output='report'))
ci.plot()

In [ ]:
# The below summary shows that the BLM Protests did have a significant negative effect on the proportion of rating 5
y1 =  list(blm_five_props.proportion)
X1 = list(avg_score_blm.score)
model_props_five = pd.DataFrame({'y':y1, 'X':X1}, columns = ['y', 'X'])
pre_period = [0,4]
post_period = [5, 11]
ci_five =CausalImpact(model_props_five, pre_period, post_period)
print(ci_five.summary())
print(ci_five.summary(output='report'))
ci_five.plot()

# After running the similar model on the covid19, 
# it seems that this shock doesn't have signifiance effect either of proportion of rating.

In [ ]:
cag_rev_pod = pd.merge(cag_rev, podcasts, on = 'Podcast_ID')
june_2020 = cag_rev_pod[(cag_rev_pod.year == 2020) & (cag_rev_pod.month == 6)]
all_cats = june_2020.groupby(['Category_x','Review_Rating'])['Podcast_ID'].count()
all_cats_props = all_cats.groupby(level = 0).apply(lambda x : 100 * x / float(x.sum()))
all_cats_props = all_cats_props.reset_index()
sorted_one = all_cats_props[all_cats_props['Review_Rating'] == 1].sort_values(by = 'Podcast_ID', ascending = False)
sorted_five = all_cats_props[all_cats_props['Review_Rating'] == 5].sort_values(by = 'Podcast_ID', ascending = False)
top_six_june = list(sorted_one.Category_x)[0:6]
top_six = june_2020[june_2020['Category_x'].isin(top_six_june)]

In [ ]:
def cat_clouds(category):
    stopwords = set(STOPWORDS)
    stopwords.update(['podcast', 'listen', 'podcasts', 'listening','episode', 'listened','s',
                      'show', 'episodes', 'used', 'good', 'love', 'listener', 'will', 'really'
                     , 'like', 'people', 'don', 't', 'm', '&','ve', 're', 'one'])

    ratings_table = top_six[(top_six.Review_Rating == 1) & (top_six.Category_x == category)]
    description_text = ' '.join(review for review in ratings_table.Review_Description)
    description_text = description_text.lower()
    description_text = description_text.strip()

    title_text = ' '.join(review for review in ratings_table.Review_Title)
    title_text = title_text.strip()
    
    print('Rating: {}'.format(1))
    print('Word Cloud for ' + category)
    plt.figure(figsize=(20, 20))
    plt.imshow(WordCloud(stopwords = stopwords).generate(description_text), interpolation = 'bilinear')
    plt.axis('off')
    plt.show()
    
    print('Word Cloud for Review Titles')
    plt.figure(figsize=(20, 20))
    plt.imshow(WordCloud(stopwords = stopwords).generate(title_text), interpolation = 'bilinear')
    plt.axis('off')
    plt.show()

In [ ]:
for cat in top_six_june:
    cat_clouds(cat)

In [ ]:
ratings_table = top_six[(top_six.Review_Rating == 1) & (top_six.Category_x == 'tv-film')]
tv_film = ratings_table.groupby('Podcast_ID')['Review_Rating'].count()
tv_film = tv_film.reset_index()
tv_film['Podcast_Name'] = [podname_finder(i)[0] for i in tv_film.Podcast_ID]
tv_film.sort_values(by = 'Review_Rating', ascending =False)

In [ ]:
pod1 = ratings_table[ratings_table['Podcast_ID'] == 'ef47233dcfdc0bf67681db6322b57f58']
pod2 = ratings_table[ratings_table['Podcast_ID'] == 'b7b4bdc0af05e557dc772b52be8cd986']

In [ ]:
pod1.head()[['Review_Title','Review_Description']]

In [ ]:
pod2.head()[['Review_Title','Review_Description']]

### Combination and Categorical Sentiment Analysis

In [ ]:
# Review with the highest positive sentiment (Same for both Titles and Descriptions)
reviews[reviews['TextBlob_Polarity_Review_Description'] == 1].head()

In [ ]:
# Review with the neutral sentiment (Same for both Titles and Descriptions)
reviews[reviews['TextBlob_Polarity_Review_Description'] == 0].tail(10)

In [ ]:
# Review Descriptions with the highest negative sentiment 
reviews[reviews['TextBlob_Polarity_Review_Description'] == -1].head()

In [ ]:
# Review Descriptions with the highest negative sentiment 
reviews[reviews['TextBlob_Polarity_Review_Title'] == -1].head()

In [ ]:
# Sentiment over the week for Descriptions
reviews['weekday'] = reviews['Review_Created_At'].dt.dayofweek
avg_sentiment = reviews.groupby(['weekday'])['TextBlob_Polarity_Review_Description'].mean().reset_index(name='Weekday Avg')
sns.lineplot(x='weekday', y='Weekday Avg', data = avg_sentiment)

In [ ]:
# Sentiment over the week for Titles
avg_sentiment = reviews.groupby(['weekday'])['TextBlob_Polarity_Review_Title'].mean().reset_index(name='Weekday Avg')
sns.lineplot(x='weekday', y='Weekday Avg', data = avg_sentiment)

In [ ]:
# Sentiment over the year for titles
avg_sentiment = reviews.groupby(['month'])['TextBlob_Polarity_Review_Description'].mean().reset_index(name='Monthly Avg')
sns.lineplot(x='month', y='Monthly Avg', data = avg_sentiment)

In [ ]:
# Sentiment over the year for titles
avg_sentiment = reviews.groupby(['month'])['TextBlob_Polarity_Review_Title'].mean().reset_index(name='Monthly Avg')
sns.lineplot(x='month', y='Monthly Avg', data = avg_sentiment)

In [ ]:
# Sentiment over the day for Descriptions
reviews['hour'] = reviews['Review_Created_At'].dt.hour
avg_sentiment = reviews.groupby(['hour'])['TextBlob_Polarity_Review_Description'].mean().reset_index(name='Hourly Avg')
sns.lineplot(x='hour', y='Hourly Avg', data = avg_sentiment)

In [ ]:
reviews['hour'] = reviews['Review_Created_At'].dt.hour
avg_sentiment = reviews.groupby(['hour'])['TextBlob_Polarity_Review_Title'].mean().reset_index(name='Hourly Avg')
sns.lineplot(x='hour', y='Hourly Avg', data = avg_sentiment)

In [ ]:
reviews = pd.merge(reviews, podcasts, on='Podcast_ID')

In [ ]:
for cat in sorted(set(reviews.Category)):
    cat_df = reviews[reviews.Category == cat]
    sentiment_analyzer(cat_df, 'Review_Description')
    sentiment_analyzer(cat_df, 'Review_Title')
    print('Category: {}'.format(cat.upper()))

    plt.bar(['Positive', 'Neutral', 'Negative'], list(cat_df['TextBlob_Analysis_Review_Description'].value_counts()), 
            color = ['orange', 'grey', 'blue'])
    plt.title('Category: {} [Review Description]'.format(cat.upper()))
    plt.ylabel('Number of Reviews')
    plt.xlabel('Sentiment for Review Descriptions')
    plt.show()
    
    plt.bar(['Positive', 'Neutral', 'Negative'], list(cat_df['TextBlob_Analysis_Review_Title'].value_counts()), 
            color = ['orange', 'grey', 'blue'])
    plt.title('Category: {} [Review Title]'.format(cat.upper()))
    plt.ylabel('Number of Reviews')
    plt.xlabel('Sentiment for Review Titles')
    plt.show()

In [ ]:
reviews['Sentiment_Combination'] = reviews[['TextBlob_Analysis_Review_Title', 'TextBlob_Analysis_Review_Description']].values.tolist()
plt.barh([str(x) for x in list(pd.DataFrame(reviews['Sentiment_Combination'].value_counts()).index)], 
         list(reviews['Sentiment_Combination'].value_counts()))
plt.title('Number of Reviews by Sentiment Combination')
plt.xlabel('Number of Reviews')
plt.ylabel('Sentiment Combination \n[Review Title Sentiment, Review Description Sentiment]')
plt.show()